# Applying Naive Bayes to N-Grams for Markov Chain Generation / Text Prediction

*Disclaimer: the code in this notebook is excerpted from the source code and is functional in and of itself; access the source code [here](https://github.com/duncanmazza/ml_ngrams).*

## Introduction and Mathematical Definitions
Many text prediction models utilize some implementation of a n-gram model where consecutive sequences of words of a specified length n in a text can used to predict the next word in a sequence. In this project, I sought to implement a n-gram-like model that utilizes a continuously generated rolling sequence of words and a graph of the directional distances between words to encode n-grams rather than explicitly use n-grams themselves. My implementation was heavily based on the work of Henrique X. Goulart, Mauro D. L. Tosi, Daniel Soares-Gonc ̧ alves, and Rodrigo F. Maiaand Guilherme Wachs-Lopes in their paper "Hybrid Model For Word Prediction Using NaiveBayes and Latent Information"[1], although significant modifications were made to appropriately scope my implementation for the weeklong project.

In the paper, the authors present a method of text prediction that combines  latent semantic analysis (LSA) and naive Bayes along with an algorithm to optimze model hyperparameters to achieve an error rate far lower than LSA or naive Bayes alone. I chose to implement my own variation of just the naive Bayes portion of their model.

### Naive Bayes Graph
Following the definition in [1], there exists a set of directional graphs $\{ G_0, G_1, ..., G_{d-1} \}$ for a vector of words $\mathbf{V}=[w_1, w_2, ..., w_d]^T$ where the vocabulary $\mathbf{V}_v$ is given by $set(\mathbf{V})$ and the nodes of graph are comprised by the vocabulary. The directional edges in graph $G_i$ between the nodes are positive integer values that represent the number of occurances of the parent node at distance $i$ from the child node in the text.

For example, in the text:
``` 
the quick brown fox jumped over the lazy dog the quick cat did not jump over the lazy dog
```
there is a distance 0 from parent node `the` to child node `quick`, and this occurs twice in the text; therefore, in graph $G_0$, the edge from `the` to `quick` has a value of 2. For another example, there is a distance 1 between parent node `jump` and child node `the` that occurs twice in the text; therefore, the edge from `jump` to `the` in graph $G_1$ has a value of 2. The utilization of this graph is discussed in the next section; following is the description of how I created it.

As a step of pre-processing for the text passed into my model, I generated and cached (in RAM) a list of graphs $[G_0, G_1, ..., G_{max\_chain}]$ where `max_chain` is a model parameter (discussed later). These graphs are stored as 2-dimensional square sparse matrices $\mathbf{M}$ of type `scscipy.sparse.dok_matrix` (dictionary of keys based sparse matrix), where indices $(i, j)$ correspond to a pair of vocabulary words $(V_{v_i}, V_{v_j})$; a dictionary called `vocab_to_matrix` stores the vocabulary words' respective index values. The matching of a vocabulary word to an index value is arbitrary as long as it is consistent across both axes and all graphs. Two nested for-loops iterate over a list of all words in $\mathbf{V}$ and over windows of size `max_chain`.


In [ ]:
# api/book.py
from scipy.sparse import dok_matrix
"""
Follows is an excerpted method of the Book class that creaates the aforementioned Bayesian graph
"""
def _make_bayesian_graphs(self):
    """
    Build the list of bayesian graphs, where the graph at index i represents the directional graph between words in
    the text vector (self.tokens), where the nodes are vocabulary words and the directional edges are the number of
    occurrences of the parent node separated by the child node by distance i in the text.
    :return:
    """
    self.graphs = []
    for i in range(self.max_chain):  # each item in self.graphs is a sparse matrix (M) where M[i, j] represents the
        # value of the directional edge from vocab_to_matrix[i] to vocab_to_matrix[j]; i and j are integers that
        # correspond to words per the dictionary vocab_to_matrix (populated below) with keys of words and values of
        # the words' corresponding indices in the matrix
        self.graphs.append(dok_matrix((self.vocabulary_size, self.vocabulary_size), dtype=int))
    self.vocab_to_matrix = {}
    for i, vocab in enumerate(self.vocabulary.keys()):
        self.vocab_to_matrix[vocab] = i  # record what matrix index corresponds to which word; values are arbitrary
        # as long as they remain unchanged and are unique to their respective key
    for t, token in enumerate(self.tokens): # iterate the search window over every word in the text
        # enable the last `self.max_chain` words in the text serve as the basis for the search window by shortening
        # the search window so that it doesn't overextend the list of words
        if self.max_chain + t >= self.num_words:
            upper_limit = self.num_words - t
        else:
            upper_limit = self.max_chain + 1
        for c in range(1, upper_limit):  # iterate over search window
            self.graphs[c - 1][self.vocab_to_matrix[token], self.vocab_to_matrix[self.tokens[c + t]]] += 1

To establish some notation for the values in this graph, let $w^d_{i, j}$ represent the occurances of word i separated from word j by distance d in the document. Note that distances are only calculated in a forward direction through the vector of word tokens.

### Naive Bayes Algorithm

The naive Bayes algorithm is generally stated as[1]:
$$
P(e|c_1, c_2,...,c_n) = \frac{P(e) \prod_{i=1}^{n}{P(c_i|e)}}{\gamma}
$$
where $e$ represents an effect that is given by of a set of causes $\{c_1, c_2, ..., c_n\}$ that are independent when conditioned on $e$, and $\gamma$ is a normalization factor.

In the model, we will consider an implementation[1] of the naive Bayes algorithm that utilizes the frequency of distances between words to calculate the likelihood term, $P(c_i|e)$, of the Bayes algorithm:
$$
B_j = P(sugg_j|\mathbf{prev}_{0:n-1}) = \frac{P(sugg_j)\prod_{i=0}^{n-1}{P^i(prev_j|sugg_i)}}{\gamma}
$$
where $\mathbf{prev}_{0:n-1}$ is the n previous words (in descending order) to the suggested word $sugg_j$, $P(sugg_j)$ is just the term frequency of $sugg_j$ in the text, and the likelihood is given by:
$$
P^i(prev_j|sugg_i) = \frac{w^i_{prev_i, sugg_j}}{\sum_{j}{w^i_{prev_i, sugg_j}}}
$$
This is interpreted as: the probability of the previous word $prev_i$ occuring at distance $i$ downstream from the suggested word $sugg_j$ is given by the number of occurances of that relationship divided by the sum of the number of occurances of $prev_j$ downstream from all suggested words. Therefore, $B_j$ is read as the probability of the suggested word given the n words preceding it, and it is calculated using the distances between the suggested and previous words in the text.

The paper[1] then goes on to integrate parameters into this expression that are later optimized; this is where I deviate from the prior work.

To account for the fact $P^i(prev_j|sugg_i)$ will often evaluate to $0$, I modified Bayes' algorithm by splitting up the product into a sum of terms using logarithms, and inserted an additive term ($\alpha$) into the logarithmic arguments to achieve Laplace smoothing:

$$
\log{B_j'} = \log{(P(sugg_j) + \alpha)} + \sum_{i=0}^{n-1}{\log{(P^i(prev_i|sugg_j) + \alpha)}}
\\
\therefore \space \space B_j' = e^{\log{(P(sugg_j) + \alpha)} + \sum_{i=0}^{n-1}{\log{(P^i(prev_i|sugg_j))}}}
$$

($B'$ is used because the probabilities are altered with the use of $\alpha$). In the case that $P^i(prev_j|sugg_i) = \infty$, I artificially set the value to 0. Note that $\gamma$ has been dropped from this equation; this is because each $B_j$ is an element of a vector $\mathbf{B'} = \frac{[B_1',B_2',...,B_d']^T}{||[B_1',B_2',...,B_d']||_1}$ which is normalized using a L1 norm such that it satisfies the property of PMFs that the sum of the probabiliies is equal to one: $||\mathbf{B'}||_1=1$. Therefore, $\gamma$ is no longer needed.

To implement this naive Bayes algorithm, I wrote several helper functions, a few of which are outlined below:
- `_p_d_i_j` - returns $P^d(i|j)$ from arguments $d$, $i$, $j$, and $\mathbf{sugg}$ (the set of suggested words in arbitrary order).
- `_p_s` - returns $P(sugg_j)$.
- `generate_cond_prob_arr` - returns $\mathbf{B}'$ (the vector of conditional probabilities) given a vector of suggested words $\mathbf{sugg}_j$ of length $k$ and a vector of previous words $\mathbf{prev}_{0:n-1}$ where $\mathbf{B}'$ is of length $k$ and $B_j'$ corresponds to the probability associated with $sugg_j$.

The suggested word with the highest probability is chosen as the next work in the Markov chain.


In [ ]:
# api/book.py
"""
Follows are excerpted methods of the Book class that implement the aforementioned functionality
"""
def _p_s(self, _s):
    """
    Returns P(s)
    :param _s: word in book vocabulary
    :return: p(s) where p(s) = (number of occurances of s) / (number of words in book)
    """
    return self.vocabulary[_s] / self.num_words

def _p_d_i_j(self, d, _p, _s, tuple_s):
    """
    Returns P^d(i, j)
    :param d: distance
    :param _p: previous word
    :param _j: suggested word
    :param tuple_s: tuple of all possible suggested words
    :return: (wight of graph d from _p to _s) / (sum of wights of graph d from _p to all list_s); if the denominator
     is 0, then return 0 to avoid dividing by 0
    """
    val_w_d_p_s = self.query_graph(d, _p, _s)
    # first, check caches for whether these values have already been queried
    val_sum_w_d_p_list_s = self.sum_w_d_p_list_s_cache.get((d, _p, tuple_s))
    if val_sum_w_d_p_list_s is None:
        val_sum_w_d_p_list_s = 0
        for _s_ in tuple_s: val_sum_w_d_p_list_s += self.query_graph(d, _p, _s_)
        self.sum_w_d_p_list_s_cache[
            (d, _p, tuple_s)] = val_sum_w_d_p_list_s  # store so that this calculation isn't redone
    if val_sum_w_d_p_list_s == 0:  # return 0 to avoid dividing by 0
        return 0
    else:
        return val_w_d_p_s / val_sum_w_d_p_list_s

def generate_cond_prob_arr(self, tuple_s, list_p_forward):
    """
    :param tuple_s: list of unique suggested words (order is arbitrary but must be maintained so words can
     correspond to values in cond_prob_arr)
    :param list_p_rev: (ordered) list of previous words preceding suggested word
    :return: numpy array of length len(list_s_set) that contains the conditional probabilities
    """
    cond_prob_arr = np.zeros((len(tuple_s)))

    # calculate create an array that has the previous words in reverse order
    list_p_rev = [list_p_forward[p] for p in range(len(list_p_forward) - 1, -1, -1)]

    # iterate over the suggested words
    for j, _s in enumerate(tuple_s):
        prior_val = self._p_s(_s)
        likelihood_arr = np.ones((len(list_p_rev))) * self.alpha  # initialize with alpha value
        for i in range(len(list_p_rev)):
            likelihood_arr[i] += self._p_d_i_j(i, list_p_rev[i], _s, tuple_s)
        log_sum_likelihood_val = np.sum(np.log(likelihood_arr))
        cond_prob_arr[j] = np.exp(np.log(prior_val + self.alpha) + log_sum_likelihood_val)

    return cond_prob_arr / np.sum(cond_prob_arr)  # normalize so values sum to 1

While the overhead to calculate the graph is non-trivial, it is a one time operation. Once that has been calculated, applying the naive Bayes algorithm in my implementation takes a trivial amount of time thanks to the fact that I cache the $\sum_{j}{w^i_{prev_i, sugg_j}} $ term so that it is only calculated once for each loop that generates another word; additionally, the implementation only consider a simple random sample of 100 suggested words if the number of suggested words is >100. Further optimizations of this algorithm would include parallelization, as calculating each $P^i(prev_i|sugg_j)$ is an independent operation.

An important consideration of this implementation is that the maximum number of previous words considered is given by a parameter `max_chains`, where the length of the previous words considered, `len(list_p)`, is equal to `max_chains`. This speeds up the execution of the algorithm, but limits the contextual inference of the algorithm to the previous `max_chain` words in a given Markov chain.

## Data Wrangling
The main routine of this code is as follows:

In [ ]:
# main.py
from api.librarian import Librarian

book_list = (
    ('Frankenstein', 'Mary Wollstonecraft (Godwin) Shelley'),
    # ('Watersprings', 'Arthur Christopher Benson')
    # ... any other book in Project Gutenberg
)

if __name__ == "__main__":
    # acquire books from the book list
    librarian = Librarian(book_list, global_truncate=0.4, global_alpha=1, global_max_chain=2)
    while True:
        for book_name in librarian.acquired_books:
            book = librarian.acquired_books[book_name]
            print("Appling naive bayes to: {}".format(book_name))
            book.apply_naive_bayes(extend_by=15)

        print('Generate another sample? (y or Y) -> ')
        while True:
            i = input()
            if i == "y" or "Y":
                break

The book list is a tuple of 2-element tuples whose first element is a book name and whose second element is the author of the book; these books must be available in the Project Gutenberg index. Initializing a Librarian object with this list prompts the "librarian" to fetch, parse, and cache (to disk) the full texts of the specified books; it does this via `Book` objects:

The `librarian` object has an attribute `acquired_books` which is a dictionary keyed by book names/authors and whose values are `Book` objects. A `Book` object upon initialization will perform a variety of parsing tasks that include tokenizing the text, storing a dictionary of vocabulary words and all possible corresponding words that occur after that word in the text, creating the aforementioned Bayesian graph, and more. The `apply_naive_bayes` method of the `Book` class is what orchestrates the generation of a Markov chain by selecting a random sample of sequential words of length `max_chains` to seed the Markov chain. The code generates a Markov chain from this seed for a specified number of iteration and prints the results.

## Results
Here is a sample of the program output with `max_chains = 15` (thus generating a seed of length 15 words) extended by 15 words:

```
Applying naive bayes to: Frankenstein, by Mary Wollstonecraft (Godwin) Shelley
--------
Seed:
"me of success with those friends whom i am on the point of meeting may..."
Generated sentence:
"me of success with those friends whom i am on the point of meeting may i remember the time that there i cursed him had received the most of divine"
Actual:
"me of success with those friends whom i am on the point of meeting may i know the names and residence of those friends i paused this i thought was"
```

Comparing the post-seed text of the generated sentence and the actual sentence taken from the book, we can see that with a short length of words, not many meaningful conclusions can be drawn about the overall behaviour of the model:

<img src="results/results_15_words.png">

Performing this same analysis on a generated sequence length of 100 words, and the results are more telling:

<img src="results/results_100_words.png">

What this chart reveals is that the model often over-predicts common words like "the" and "and" compared to the actual text, and it predicts many words that are not in the actual sentence; this is to be expected, as any deviation from the actual text is compounded the longer the Markov chain is. An important success of the model is that it follows the overal-trend of word frequency in the document: because the x-axis labels are ordered from most-common in the whole text body to least-common in the whole text body from left to right respectively, we would expect the above histograms to be right-skewed - and indeed they are.

Adjusting the `max_chains` paramter to a value of 2 (effectively simulating bi-grams), the tendancy to over-predict common words is mitigated, but results in more spikes in the graph for the less-common words. Qualitatively (and in my opinion), the results with `max_chains=2` are less coherent:

<img src="results/results_100_words_max_chain_2.png">

```
Applying naive bayes to: Frankenstein, by Mary Wollstonecraft (Godwin) Shelley
--------
Seed:
"which fell..."
Generated sentence:
"which fell however finding that and hunger when unprejudiced by an interview with contempt yet taken possession"
Actual:
"which fell into her hands she heard of the exile of her lover and learnt the name"
```

## Future Work
My first course of action to improve upon this project would be to implement unit testing to ensure that all of the probabilities are being calculated correctly; while I manually probed data with my debugger to ensure on a preliminary level that probabilities were being calculated correctly, it was not as rigorous as fully-implemented unit testing.

Following this, a good next step would be integrating this functionality with optimized model hyperparameters (per [1]).

[1] https://arxiv.org/pdf/1803.00985.pdf